# Nevenfuncties ophalen bij Rechtspraak.nl

In [1]:
import json

import pandas as pd
import requests as req

from time import sleep

from ipywidgets import Text
from ipywidgets import IntProgress
from IPython.display import display

from lxml import html

In [20]:
crawl_delay = 0.25;

Er zijn een paar URLS die we nodig hebben voor deze klus:

In [2]:
instanties = 'https://namenlijst.rechtspraak.nl/Services/ConfigService/getInstanties'
index = "https://namenlijst.rechtspraak.nl/#!/zoeken/index"
search = "https://namenlijst.rechtspraak.nl/Services/WebNamenlijstService/Zoek"

Omdat de site met sessies werkt, kunenn we geen enkele requests doen, maar moeten we met een sessie werken. Eerst een sessie maken dus!

In [3]:
client = req.session() # a session to take care of CSRF cookies

Vervolgens maar eens een lijst van de instanties ophalen

In [4]:
d = client.get(instanties).json()

Krijgen we een mooie lijst terug van waar we naar kunnen zoeken...

In [50]:
d['instanties'][0]['naam'] # hoge raad
d['instanties'][1]['naam'] # Centrale Raad van Beroep
d['instanties'][2]['naam'] # College van Beroep voor het bedrijfsleven
d['instanties'][3]['naam'] # gerechtshoven
d['instanties'][3]['locaties'] # lijst van locaties van gerechtshoven
d['instanties'][3]['locaties'][0] # {'code': 'RS0055', 'naam': 'Amsterdam'}
d['instanties'][3]['locaties'][1] # {'code': 'RS0056', 'naam': 'Arnhem-Leeuwarden'}
d['instanties'][3]['locaties'][2] # {'code': 'RS0057', 'naam': 'Den Haag'}
d['instanties'][3]['locaties'][2] # {'code': 'RS0058', 'naam': "'s-Hertogenbosch"}
d['instanties'][4]['naam'] # 'Rechtbanken'
d['instanties'][4]['locaties'] # lijst van locaties van rechtbanken
d['instanties'][4]['locaties'][0] # {'code': 'AR0040', 'naam': 'Amsterdam'}
d['instanties'][4]['locaties'][1] # {'code': 'AR0041', 'naam': 'Noord-Holland'}
d['instanties'][4]['locaties'][2]#{'code': 'AR0042', 'naam': 'Midden-Nederland'}
d['instanties'][4]['locaties'][3]#{'code': 'AR0043', 'naam': 'Noord-Nederland'}
d['instanties'][4]['locaties'][4]#{'code': 'AR0045', 'naam': 'Den Haag'}
d['instanties'][4]['locaties'][5]#{'code': 'AR0046', 'naam': 'Rotterdam'}
d['instanties'][4]['locaties'][6]#{'code': 'AR0047', 'naam': 'Limburg'}
d['instanties'][4]['locaties'][7]#{'code': 'AR0048', 'naam': 'Oost-Brabant'}
d['instanties'][4]['locaties'][8]#{'code': 'AR0049', 'naam': 'Zeeland-West-Brabant'}
d['instanties'][4]['locaties'][9]#{'code': 'AR0050', 'naam': 'Gelderland'}
d['instanties'][4]['locaties'][10]# {'code': 'AR0051', 'naam': 'Overijssel'}

{'code': 'AR0051', 'naam': 'Overijssel'}

om die instanties af te zoeken wil de site een "model", Hieronder zie je hoe zo'n request eruit ziet voor de verschillende typen instanties.

In [ ]:
# JSON request voor overige instellingen: {"model":"{\"instantieCode\":[],\"overige_instantie\":[\"Centrale Raad van Beroep\"],\"naam\":\"\",\"functie\":\"\"}"}
# JSON request voor gerechtshoven: {"model":"{\"instantieCode\":[\"RS0055\"],\"overige_instantie\":[],\"naam\":\"\",\"functie\":\"\"}"}
# JSON request voor rechtbanken: {"model":"{\"instantieCode\":[\"AR0040\"],\"overige_instantie\":[],\"naam\":\"\",\"functie\":\"\"}"}

laten we hieronder maar even een lijst maken van alle mogelijke modellen op basis van de informatie die we net over de instanties gekregen hebben.

In [12]:
d['instanties'][3]['locaties'][0]['code']

'RS0055'

In [17]:
l = []

for i in range(len(d['instanties'])):
   
    if i < 3:
        l.append({"model":"{\"instantieCode\":[],\"overige_instantie\":[\""+d['instanties'][i]['naam'] +"\"],\"naam\":\"\",\"functie\":\"\"}"})
    else:
        for j in range(len(d['instanties'][i]['locaties'])):
            l.append({"model":"{\"instantieCode\":[\""+d['instanties'][i]['locaties'][j]['code']+"\"],\"overige_instantie\":[],\"naam\":\"\",\"functie\":\"\"}"})
    

ok, tijd om maar eens wat requests naar de search service te gooien... De search service werkt alleen als we een geldige sessie hebben, met een cookie en een CSRF Requesttoken die we uit de HTML moeten vissen...

In [6]:
r = client.get(index)
tree = html.fromstring(r.content)
token = tree.xpath('//input[@name="__RequestVerificationToken"]/@value')[0] # retrieve CSRF token from HTML

Die token, moeten we dus zometeen weer meesturen in de headers van onze search, even bewaren voor zometeen dus...

In [7]:
headers = {'__RequestVerificationToken':token}

In [26]:
max_count = len(l)-1

text_out = Text(value='default value', placeholder='https://www.rechtspraak.nl', description='current URL:', disabled=False,layout={'width': 'auto'}) # instantiate the progress bar
progress = IntProgress(min=0, max=max_count,layout={'width':'auto'}) # instantiate the progress bar

display(text_out) # display the text
display(progress) # display the bar

for idx in range(len(l)):
    request_data = l[idx]
   
    url = 'https://namenlijst.rechtspraak.nl/Services/WebNamenlijstService/Zoek'
    text_out.value = url
    progress.value = idx
    progress.description = f'[{idx}/{max_count}]'

    r = client.post(url,json=request_data, headers=headers)
    with open(f'zoekinstantie/zoek-instantie-{idx}.json', 'wb') as f:
            f.write(r.content)
    
    sleep(crawl_delay)
    

Text(value='default value', description='current URL:', layout=Layout(width='auto'), placeholder='https://www.…

IntProgress(value=0, layout=Layout(width='auto'), max=18)